## Core Fisici vs Thread Logici

- **Core Fisico**: Hardware reale dentro la CPU.
  * Unità di elaborazione fisica
  * Ha proprie ALU (Arithmetic Logic Unit), registri, cache L1/L2
  * Esegue istruzioni indipendentemente dagli altri core

  4 core = 4 unità di calcolo fisiche separate.

- **Thread Logici**: Astrazione software che sfrutta risorse del core.
  * "Flussi di esecuzione" che il core può gestire
  * Con Hyper-Threading (tecnologia Intel): 1 core fisico = 2 thread logici

- **Hyper-Threading**: Il core ha risorse duplicate (registri, program counter)
Quando thread A si blocca (es. attende memoria), il core switcha istantaneamente a thread B

- Core fisico: cucina con fornelli, forno, piano di lavoro (Risorse)
- Hyper-Threading: 1 cucina = 2 cuochi che condividono attrezzature

Quando cuoco A aspetta che l'acqua bolla, cuoco B usa il piano di lavoro. Entrambi lavorano, ma condividono le risorse fisiche.

---

# Job System

Il Job System ottimizza l’uso della CPU trasformando il multitasking pesante in una gestione agile di piccoli carichi di lavoro:

- **Worker Thread e Core Logici**: Invece di creare thread arbitrari, il sistema ne alloca uno per ogni core logico della CPU, sfruttando l'hardware al 100% senza sovraccaricarlo.
- **Dai Task alla Job Queue**: Piuttosto che assegnare un intero compito (task) a un thread specifico, i lavori vengono frammentati in piccoli Job racchiusi in una coda (queue).
- **Efficienza Operativa**: I Worker Thread "pescano" continuamente dalla coda. Questo elimina il costo di gestione dei singoli thread, riduce lo spreco di risorse e permette di eseguire una mole enorme di calcoli in parallelo.

### RIASSUMENDO
I Job System lavorano in un Worker Thread, ma essenziale per 1 core logico...ma la positività dei job system è che sono semplicemente piccoli e racchiusi in una coda "queque". Quindi piuttosto che chiamare "task" per thread, raggruppiamo tutte quelle task in una coda, in un worker thread...quelle task vengono pescate dal pool. 

---

### 1. Obiettivo e Vantaggi

Il **C# Job System** permette di scrivere codice **multithreaded** sicuro e performante in Unity.

* **Performance:** Aumenta il frame rate e, combinato con il **Burst Compiler**, riduce il consumo di batteria (codice macchina ottimizzato).
* **Integrazione:** Si integra nativamente con il sistema interno di Unity, condividendo gli stessi **Worker Threads** per evitare conflitti sulla CPU.

### 2. Architettura: Worker Threads vs Thread Classici

* **Problema dei Thread Classici:** Creare un thread per ogni piccola operazione causa overhead e **Context Switching** (la CPU perde tempo a salvare/caricare stati invece di lavorare), saturando il sistema.
* **Soluzione Job System:** Usa un numero fisso di **Worker Threads** (solitamente uno per core logico della CPU). I "Job" (piccoli compiti) vengono messi in una **Job Queue**. I Worker prelevano i job dalla coda ed eseguono il lavoro in sequenza, mantenendo la CPU sempre attiva senza sprechi.

### 3. Sicurezza e Race Conditions

Il sistema protegge dai **Race Conditions** (bug causati da thread che accedono agli stessi dati contemporaneamente con tempistiche imprevedibili).

* **Isolamento:** I Job lavorano su **copie** dei dati, non sui riferimenti originali (usando tipi *blittable*).
* **Dipendenze:** Il sistema gestisce l'ordine di esecuzione. Se il Job A prepara dati per il Job B, il sistema assicura che B parta solo dopo che A ha finito.

### 4. NativeContainer: Memoria Condivisa

Poiché i Job usano copie dei dati, per riportare i risultati al thread principale è necessario usare i **NativeContainer**.

* Sono wrapper C# per la memoria nativa (non gestita dal Garbage Collector standard).
* Esempi: `NativeArray`, `NativeList`, `NativeQueue`.
* **Sicurezza Integrata:** Includono controlli (es. `AtomicSafetyHandle`) per prevenire errori di lettura/scrittura simultanea. L'attributo `[ReadOnly]` permette a più job di leggere lo stesso dato contemporaneamente in sicurezza.

### 5. Gestione della Memoria (Allocators)

Quando crei un NativeContainer, devi specificare quanto deve durare la memoria:

1. **Allocator.Temp:** Velocissimo, dura **1 frame**. (Non usabile nei Job).
2. **Allocator.TempJob:** Veloce, dura fino a **4 frame**. (Standard per la maggior parte dei Job piccoli).
3. **Allocator.Persistent:** Più lento, dura per tutta la vita dell'app (simile a `malloc`).

---


# Creazione di Job in Unity (Job System)

### Definizione

Un **Job** è una `struct` che implementa l'interfaccia `IJob`. Questo costrutto permette di eseguire codice in un thread separato (worker thread) parallelamente al Main Thread di Unity.

### Workflow di Creazione

Per implementare un Job funzionante, è necessario seguire tre passaggi fondamentali:

#### 1. Definizione della Struct

È necessario dichiarare una `struct` (non una classe) e implementare l'interfaccia `IJob`.

```csharp
public struct MyJob : IJob
{
    // ...
}

```

#### 2. Variabili Membro (Dati)

I Job possono contenere esclusivamente tipi di dati "Thread-Safe":

* **Blittable Types** (Dati copiati per valore):
* Includono `int`, `float`, `bool`, `double`, ecc.
* Altre `struct` composte esclusivamente da tipi blittable.
* Questi dati vengono copiati nel job; le modifiche non si riflettono all'esterno.


* **NativeContainers** (Dati condivisi per riferimento):
* Includono `NativeArray<T>`, `NativeList<T>`, ecc.
* Sono indispensabili per scrivere i risultati dell'elaborazione e renderli accessibili al Main Thread.

```csharp
// Esempio di dichiarazione dati
public float a;                   // Input (copia per valore)
public float b;                   // Input (copia per valore)
public NativeArray<float> result; // Output (riferimento alla memoria condivisa)

```

#### 3. Metodo Execute()

Il metodo `Execute()` rappresenta il punto di ingresso dove risiede la **logica del job**.

```csharp
public void Execute()
{
    // Esegue la logica nel worker thread
    result[0] = a + b;
}

```

### Dettagli sull'Esecuzione

Il metodo `Execute()` di un `IJob` standard viene eseguito **una volta** su **un singolo core** (un singolo thread worker).

Se l'obiettivo è elaborare molti elementi in parallelo su più core contemporaneamente, è necessario utilizzare varianti specifiche come `IJobParallelFor`.

### Esempio Completo e Analisi

```csharp
using Unity.Jobs;
using Unity.Collections;

public struct MyJob : IJob
{
    public float a;                       // Input: copiato nel job
    public float b;                       // Input: copiato nel job
    public NativeArray<float> result;     // Output: memoria condivisa

    public void Execute()
    {
        result[0] = a + b;                // Scrittura risultato in memoria condivisa
    }
}

```

#### Flusso dei Dati

1. **Copia:** Le variabili `a` e `b` vengono copiate nel job e isolate dal Main Thread.
2. **Calcolo:** Il job calcola la somma.
3. **Scrittura:** Il risultato viene scritto in `result[0]`. Essendo un `NativeArray`, questa memoria è condivisa.
4. **Lettura:** Il Main Thread può leggere `result[0]` solo dopo il completamento del job.

**Pattern Chiave:**

* **Input:** Blittable types (Copie sicure).
* **Output:** NativeContainer (Memoria condivisa necessaria per estrarre i dati).

### Architettura Unity con Job System

Per comprendere dove girano i job, è utile distinguere i ruoli dei thread:

**Main Thread (Unico):**

* Gestisce Rendering, Input e Callback della Fisica.
* Gestisce il ciclo di vita dei MonoBehaviour (Start, Update, ecc.).
* Gestisce la UI.
* Si occupa dello Scheduling dei job e della lettura dei risultati.

**Worker Threads (Multipli, es. 7-8 core):**

* Eseguono esclusivamente i Job.
* Non hanno accesso ai GameObjects o ai Transform.
* Non possono chiamare le API di Unity.
* Lavorano su dati isolati.

### Design Pattern: Job + Executor

Per mantenere il codice pulito e scalabile, si adotta la separazione tra logica di calcolo e orchestrazione.

#### 1. Il "Cervello" (Logica Pura)

* **File:** `MyCalculationJob.cs` (Struct `IJob` o `IJobParallelFor`).
* **Thread:** Worker Thread.
* **Funzione:** Contiene solo dati (`NativeArray`) e matematica pura.
* **Vincolo:** Nessun riferimento a `UnityEngine` (niente `transform`, `GameObject`, ecc.).

#### 2. Il "Manager" (Orchestrazione)

* **File:** `JobExecutor.cs` (Classe `MonoBehaviour`).
* **Thread:** Main Thread (solitamente in `Update`).
* **Funzione:** Gestisce l'intero ciclo di vita del job:
1. **Allocazione:** Crea i `NativeArray` per input e output.
2. **Schedule:** Istanzia la struct Job e chiama `.Schedule()`.
3. **Complete:** Chiama `.Complete()` per sincronizzare i thread.
4. **Dispose:** Libera obbligatoriamente la memoria dei `NativeArray` per evitare memory leak.

**Sintesi:** Uno script esegue i calcoli (Job), l'altro gestisce le risorse e i tempi (MonoBehaviour).

---

# Miglioramenti Performance con Job System

| **Aspetto** | **Senza Jobs** | **Con Jobs** | **Guadagno** |
|-------------|----------------|--------------|--------------|
| **Tempo esecuzione** | 10ms (1 core) | 1.5-2ms (8 cores) | **5-7x più veloce** |
| **Frame rate** | 55 FPS | 100+ FPS | **+45 FPS** |
| **Utilizzo CPU** | 1 core 100%, 7 idle | 8 cores ~50% | **8x meglio distribuito** |
| **Entità gestibili** | 1000 nemici → frame drop | 5000 nemici → fluido | **5x scalabilità** |
| **Consumo batteria** | Alto (1 core boost) | Ridotto 30-40% | **-35% consumo** |
| **Margine frame** | 1.4ms liberi | 8ms liberi | **+6.6ms per features** |

## Formula Rapida

```
Speedup ≈ Numero Cores × 0.75

```

---

# Scheduling Jobs - Sintesi

### Flusso di Esecuzione

1. **Main Thread**: Istanzia il job e popola i dati.
2. **Main Thread**: Chiama `Schedule()`.
3. **Queue**: Il job entra in coda.
4. **Worker Thread**: Preleva il job ed esegue `Execute()` appena disponibile.

**Regole:**

* `Schedule()` deve essere chiamato solo dal Main Thread.
* Una volta schedulato, il job non può essere interrotto.

### Esempio di Implementazione

```csharp
// 1. Allocazione memoria condivisa (Input/Output)
NativeArray<float> result = new NativeArray<float>(1, Allocator.TempJob);

// 2. Setup Dati
MyJob jobData = new MyJob();
jobData.a = 10;
jobData.b = 10;
jobData.result = result; // Passaggio per riferimento

// 3. Scheduling (Ritorna JobHandle)
JobHandle handle = jobData.Schedule();

// 4. Sincronizzazione (Il Main Thread attende la fine del job)
handle.Complete();

// 5. Lettura Risultati (Sicuro dopo Complete)
float total = result[0];

// 6. Pulizia Memoria (Obbligatorio per evitare leak)
result.Dispose();

```

### JobHandle e Sincronizzazione

Il `JobHandle` è il riferimento al job in coda.

* **Complete()**: Blocca il Main Thread finché il job non è terminato. È necessario chiamarlo prima di leggere i risultati o fare il `Dispose` dei `NativeArray`.

### Timeline Semplificata

```text
Main Thread:   Crea Dati -> Schedule() -> [Altro Lavoro] -> Complete() (Attesa) -> Leggi Dati -> Dispose()
Worker Thread:                                           -> Esegue Job -> Fine

```

### Differenza Schedule vs Run

* **Schedule()**: Esecuzione asincrona su Worker Thread (Standard per produzione).
* **Run()**: Esecuzione sincrona immediata su Main Thread (Utile solo per debug).

### Regola d'Oro (Lifecycle)

L'ordine delle operazioni è rigido:

1. **Allocate** (NativeArray)
2. **Schedule** (Job)
3. **Complete** (JobHandle)
4. **Dispose** (NativeArray)

---

# JobHandle e Dipendenze (Job Dependencies)

### Definizione

Il **JobHandle** è un riferimento (struct) restituito dal metodo `Schedule()`. Rappresenta un job in coda e serve a:

1. Controllare lo stato del job.
2. Costruire catene di dipendenze.
3. Sincronizzare il job con il Main Thread (`Complete()`).

### Gestione delle Dipendenze

Le dipendenze permettono di definire l'ordine di esecuzione: il **Job B** inizia solo dopo che il **Job A** è terminato.

#### 1. Dipendenza Sequenziale (Catena)

Per creare una dipendenza, si passa l'`handle` del primo job come argomento nello `Schedule` del secondo.

```csharp
JobHandle handleA = jobA.Schedule();        // Job A parte
JobHandle handleB = jobB.Schedule(handleA); // Job B aspetta Job A

```

#### 2. Dipendenze Multiple (Fan-In)

Se un job deve attendere il completamento di più job paralleli (es. Job C aspetta A, B e D), si utilizza `CombineDependencies`.

```csharp
NativeArray<JobHandle> handles = new NativeArray<JobHandle>(3, Allocator.TempJob);
handles[0] = handleA;
handles[1] = handleB;
handles[2] = handleD;

JobHandle combined = JobHandle.CombineDependencies(handles);
JobHandle handleC = jobC.Schedule(combined); // C parte solo quando A, B e D finiscono

handles.Dispose(); // Ricordare di liberare l'array degli handle

```

### Il ruolo di Complete() e Safety System

Il metodo `Complete()` svolge tre funzioni critiche:

1. **Sincronizzazione:** Blocca il Main Thread finché il job non è terminato.
2. **Propagazione:** Chiamare `Complete()` sull'ultimo handle di una catena (es. `handleB`) completa automaticamente anche tutte le dipendenze a monte (es. `handleA`).
3. **Ownership Transfer:** Restituisce la "proprietà" dei `NativeContainer` dal Worker Thread al Main Thread. Senza `Complete()`, il Main Thread non può leggere o scrivere i dati in sicurezza (il Safety System lancerà un'eccezione).

### Esecuzione Forzata (Batching)

I job schedulati non partono istantaneamente, ma vengono raggruppati (batched) per efficienza.

* **`JobHandle.ScheduleBatchedJobs()`**: Forza l'invio immediato dei job ai worker thread.
* **Utilizzo**: Raramente necessario. Usare solo se si nota latenza eccessiva nell'avvio dei job, poiché rompe le ottimizzazioni di batching.

### Pattern Comune: La Pipeline

```csharp
// Setup Pipeline: A (Calcolo) -> B (Elaborazione) -> C (Output)
JobHandle hA = jobA.Schedule();
JobHandle hB = jobB.Schedule(hA);
JobHandle hC = jobC.Schedule(hB);

// Esecuzione
hC.Complete(); // Aspetta A, B e C

// Lettura risultati
// ...
// Dispose memoria

```

**Regola Chiave:** Le dipendenze costruiscono un grafo di esecuzione sicuro. Unity garantisce che un job non acceda ai dati finché il job da cui dipende non ha finito di scriverli.

```text
Main Thread Schedula:
├─ jobA.Schedule()        → entra in QUEUE
├─ jobB.Schedule(handleA) → entra in QUEUE (ma con flag "aspetta A")
└─ jobC.Schedule(handleB) → entra in QUEUE (ma con flag "aspetta B")

Job Queue:
[jobA] [jobB - aspetta A] [jobC - aspetta B]
   ↓
Worker Thread 1 prende jobA
   ↓
Execute jobA
   ↓
jobA completa → jobB SBLOCCATO
   ↓
Worker Thread 2 prende jobB
   ↓
Execute jobB
   ↓
jobB completa → jobC SBLOCCATO
   ↓
Worker Thread 3 prende jobC
   ↓
Execute jobC
   ↓
Main Thread: handleC.Complete() ritorna
```

## Job Queue + Dependencies = Sequenza Controllata

### Visualizzazione Completa

```
Main Thread Schedula:
├─ jobA.Schedule()        → entra in QUEUE
├─ jobB.Schedule(handleA) → entra in QUEUE (ma con flag "aspetta A")
└─ jobC.Schedule(handleB) → entra in QUEUE (ma con flag "aspetta B")

Job Queue:
[jobA] [jobB - aspetta A] [jobC - aspetta B]
   ↓
Worker Thread 1 prende jobA
   ↓
Execute jobA
   ↓
jobA completa → jobB SBLOCCATO
   ↓
Worker Thread 2 prende jobB
   ↓
Execute jobB
   ↓
jobB completa → jobC SBLOCCATO
   ↓
Worker Thread 3 prende jobC
   ↓
Execute jobC
   ↓
Main Thread: handleC.Complete() ritorna
```

---

## Due Modalità di Esecuzione + 1

### 1. Sequenziale (con Dependencies)

```csharp
// SEQUENCE: A poi B poi C
JobHandle handleA = jobA.Schedule();
JobHandle handleB = jobB.Schedule(handleA);  // ← aspetta A
JobHandle handleC = jobC.Schedule(handleB);  // ← aspetta B

handleC.Complete();  // Aspetta tutta la catena
```

**Timeline**:
```
Worker: |---A---|---B---|---C---|
         2ms     2ms     2ms
Totale: 6ms (sequenziale)
```

**Quando usare**: quando **job B SERVE i risultati di job A** (dipendenza dati reale).


### 2. Parallelo (senza Dependencies)

```csharp
// PARALLEL: A, B, C simultaneamente
JobHandle handleA = jobA.Schedule();
JobHandle handleB = jobB.Schedule();  // ← NO dipendenza
JobHandle handleC = jobC.Schedule();  // ← NO dipendenza

// Aspetta tutti (ma girano in parallelo)
handleA.Complete();
handleB.Complete();
handleC.Complete();
```

**Timeline**:
```
Worker 1: |---A---|
Worker 2: |---B---|
Worker 3: |---C---|
           2ms (tutti insieme)
Totale: 2ms (parallelo)
```

**Quando usare**: quando **A, B, C sono indipendenti** (nessuna dipendenza dati).

## 3. Pattern: Mix Sequenziale + Parallelo

```csharp
// FASE 1: Due jobs indipendenti (paralleli)
JobHandle handleA = jobA.Schedule();
JobHandle handleB = jobB.Schedule();

// FASE 2: Job C aspetta ENTRAMBI A e B
NativeArray<JobHandle> deps = new NativeArray<JobHandle>(2, Allocator.Temp);
deps[0] = handleA;
deps[1] = handleB;
JobHandle combined = JobHandle.CombineDependencies(deps);

JobHandle handleC = jobC.Schedule(combined);

handleC.Complete();
deps.Dispose();
```

**Timeline**:
```
Worker 1: |---A---|
Worker 2: |---B---|
            ↓ (entrambi finiti)
Worker 3:       |---C---|
```

**Beneficio**: parallelizza dove possibile, sequenza dove necessario.

---

## Analogia Finale: Catena di Montaggio

**Senza Dependencies (parallelo)**:
```
3 operai lavorano su 3 auto diverse contemporaneamente
→ veloce, ma serve che siano auto indipendenti
```

**Con Dependencies (sequenziale)**:
```
Operaio 1: monta motore
    ↓ (finisce)
Operaio 2: monta ruote (SERVE il motore già montato)
    ↓ (finisce)
Operaio 3: vernicia (SERVE ruote già montate)
```

**Dependencies = ordine garantito nella catena di montaggio.**

---

## Tabella Riassuntiva

| **Scenario** | **Codice** | **Esecuzione** |
|--------------|-----------|----------------|
| Jobs indipendenti | `Schedule()` senza param | **Parallelo** |
| Jobs dipendenti | `Schedule(handlePrecedente)` | **Sequenziale** |
| Mix | `CombineDependencies()` | **Parallelo poi sequenziale** |
| Sincronizzazione main | `handle.Complete()` | Main thread **aspetta** |

---



<!-- ### Cos'è una Race Condition

Output di un'operazione dipende dal timing di un altro processo.

'''text
Main Thread: legge variabile X
Job Thread: scrive variabile X

Chi arriva prima? → Risultato imprevedibile
''' -->